In [ ]:
import sys
# change the path to the path of codes/models
sys.path.append(PATH_TO_codes_models)
import os
import pandas as pd
import numpy as np
from llm_interface import LLMFactory
import re
import time
import nest_asyncio
nest_asyncio.apply()

In [2]:
schwartz_value_description = """For the following task, you can reference the following list of Schwartz values and their definitions:
1. Self-direction - independent thought and action-choosing, creating, exploring
2. Stimulation - excitement, novelty and challenge in life
3. Hedonism - pleasure or sensuous gratification for oneself
4. Achievement - personal success through demonstrating competence according to social standards
5. Power - social status and prestige, control or dominance over people and resources
6. Security - safety, harmony, and stability of society, relationships, and of self
7. Conformity - restraint of actions, inclinations, and impulses likely to upset or harm others and violate social expectations or norms
8. Tradition - respect, commitment, and acceptance of the customs and ideas that one's culture or religion provides
9. Benevolence - preserving and enhancing the welfare of those with whom one is in frequent personal contact (the 'in-group')
10. Universalism - understanding, appreciation, tolerance, and protection for the welfare of all people and for nature
"""
schwartz_value_defination = {
	'Self-direction': 'independent thought and action-choosing, creating, exploring',
	'Stimulation': 'excitement, novelty and challenge in life',
	'Hedonism': 'pleasure or sensuous gratification for oneself',
	'Achievement': 'personal success through demonstrating competence according to social standards',
	'Power': 'social status and prestige, control or dominance over people and resources',
	'Security': 'safety, harmony, and stability of society, relationships, and of self',
	'Conformity': 'restraint of actions, inclinations, and impulses likely to upset or harm others and violate social expectations or norms',
	'Tradition': "respect, commitment, and acceptance of the customs and ideas that one's culture or religion provides",
	'Benevolence': 'preserving and enhancing the welfare of those with whom one is in frequent personal contact (the in-group)',
	'Universalism': 'understanding, appreciation, tolerance, and protection for the welfare of all people and for nature'
}

big5_defination = {"Conscientiousness": "Conscientiousness in the Big Five Inventory relates to impulse control, organization, and goal-directed behavior. It differentiates disciplined, reliable individuals from those who are disorganized. Key traits include self-control, delaying gratification, and avoiding impulsiveness, which can lead to negative outcomes.\nThe six facets of conscientiousness are:\n1. Competence: Capability and effectiveness.\n2. Order: Organization and methodical approach.\n3. Dutifulness: Moral adherence and reliability.\n4. Achievement Striving: Ambition and diligence.\n5. Self-Discipline: Task initiation and persistence.\n6. Deliberation: Thoughtful decision-making.\nThese facets contrast high scorers, demonstrating strong traits, with low scorers, showing opposite tendencies like disorganization and impulsivity.",
			"Openness": "Openness in the Big Five Inventory relates to a cognitive style that values exploration and appreciation of new experiences. It differentiates intellectually curious, creative individuals from those who are traditional and closed-minded. Openness involves a preference for abstract over concrete thinking and a tendency towards novelty rather than convention.\nThe six facets of openness are:\n1. Fantasy: Active imagination and vivid fantasy life.\n2. Aesthetics: Deep appreciation for art and beauty.\n3. Feelings: Sensitivity to, recognition, and valuing of one's own emotions.\n4. Actions: Willingness to try new experiences and embrace change.\n5. Ideas: Intellectual curiosity and openness to unconventional ideas.\n6. Values: Reexamination of social, political, and religious values, challenging tradition and authority.\nThese facets highlight a contrast between high scorers, who display strong openness traits, and low scorers, who exhibit more conventional, practical thinking.",
			"Agreeableness": "Agreeableness in the Big Five Inventory assesses an individual's likability and attitudes towards others, balancing compassion and sympathy with antagonism and distrust. It encapsulates a broad interpersonal orientation, emphasizing cooperation and social harmony.\nThe six facets of agreeableness are:\n1. Trust: Belief in others' honesty and good intentions.\n2. Straightforwardness: Frankness and sincerity, contrasting with manipulative tendencies.\n3. Altruism: Generosity and willingness to assist others.\n4. Compliance: Preference for harmony over conflict, with a tendency to be accommodating.\n5. Modesty: Humbleness and self-effacement, as opposed to arrogance.\n6. Tender-mindedness: Sympathy and concern for others, versus a more hardheaded and objective approach.\nHigh scorers in agreeableness are seen as good-natured, cooperative, and trusting, whereas low scorers may prioritize self-interest, be indifferent to others, and exhibit skepticism towards people's motives.",
			"Extraversion": "Extraversion in the Big Five Inventory measures the quantity and intensity of interpersonal interaction, need for stimulation, and capacity for joy, contrasting social, outgoing individuals with reserved, shy types. It's evaluated through interpersonal involvement and activity level.\nThe six facets of extraversion are:\n1. Warmth: Affection and friendliness, with high scorers enjoying close relationships.\n2. Gregariousness: Preference for company, with high scorers enjoying lively settings.\n3. Assertiveness: Social dominance, with high scorers often becoming leaders.\n4. Activity: Pace of life, with high scorers leading fast-paced, busy lives.\n5. Excitement Seeking: Craving for stimulation, with high scorers seeking thrills.\n6. Positive Emotions: Tendency to experience joy and optimism.\nExtraverted people are energetic, enjoy interaction, and often feel positive emotions. They are enthusiastic and seek excitement. Introverted individuals are quieter, cautious, and value solitude, often misunderstood as unfriendly or arrogant, but can be kind and approachable.",
			"Neuroticism": "Neuroticism, or Emotional Stability, in the Big Five Inventory, refers to tendencies towards anxiety, hostility, depression, self-consciousness, impulsiveness, and vulnerability. It contrasts individuals who are prone to emotional instability with those who are emotionally stable and less reactive to stress.\nThe six facets of neuroticism are:\n1. Anxiety: High levels indicate a tendency to worry and feel tense.\n2. Angry Hostility: Reflects the ease of experiencing anger and frustration.\n3. Depression: Measures the tendency to feel sadness, hopelessness, and loneliness.\n4. Self-Consciousness: Involves shyness and sensitivity to social scrutiny.\n5. Impulsiveness: Indicates difficulty in controlling cravings and urges.\n6. Vulnerability: Measures susceptibility to stress and difficulty coping.\nHigh scorers in neuroticism may experience more negative emotions like fear, anger, and sadness, and find it hard to cope with stress. Low scorers tend to be calmer, more relaxed, and emotionally stable, handling stress and social situations more effectively."
}

big5_value_description = """For the following task, you can reference the following list of Big Five personality traits and their definitions:\n""" + "\n".join([f"{i+1}. {trait} - {big5_defination[trait]}" for i, trait in enumerate(big5_defination.keys())])


moral_foundation_description = """For the following task, you can reference the following list of moral foundations and their definitions:
1. Care/Harm - cherishing and protecting others, and empathizing with those who suffer
2. Fairness/Cheating - rendering justice according to shared rules, and avoiding cheating
3. Loyalty/Betrayal - standing with your group, family, nation, or tribe
4. Authority/Subversion - obeying tradition and legitimate authority, and respecting those in power
5. Sanctity/Degradation - avoiding disgusting things, which are seen as unworthy of respect and protection
"""
moral_foundation_defination = {
	'Care/Harm': 'cherishing and protecting others, and empathizing with those who suffer',
	'Fairness/Cheating': 'rendering justice according to shared rules, and avoiding cheating',
	'Loyalty/Betrayal': 'standing with your group, family, nation, or tribe',
	'Authority/Subversion': 'obeying tradition and legitimate authority, and respecting those in power',
	'Sanctity/Degradation': 'avoiding disgusting things, which are seen as unworthy of respect and protection'
}

def get_prompt(dimension, description, case_number=120, max_length=50):
	return f"""{description}

	In the following task, you should try to write {case_number} diverse demonstration sentences, each of which contains a reflection and an action comparison in actual scenario, based on a single given dimention, here is an template:
	[Dimension]: <text of the dimension>
	1. [Reflection], e.g.: [Action Comparison]
	2. ...

	And here are two examples:
	[Dimension]: Openness
	1. I am curious and eager to learn new things, e.g.: I often find myself researching topics that I'm not familiar with, just for the sake of expanding my knowledge.
	2. I enjoy exploring new cultures and ways of life, e.g.: I would rather travel to a country I've never been to before than revisit a familiar destination.
	3. ...

	[Dimesion]: Henonism
	1. I enjoy indulging in pleasurable activities, e.g.: I spend my weekend on spa rather than working overtime.
	2. I seek out experiences that bring me joy, e.g.: I prefer to go to a concert than stay at home watching TV.
	3. ...

	Each line should not exceed {max_length} words. Please make sure that the sentences are coherent and meaningful. 
	Now, please write down {case_number} demonstration sentences for the given dimension: {dimension}.
	Your response: """

def extract_demonstration(response):
	# 1. ... \n
	pattern = re.compile(r'\d+\.\s(.*?)\n')
	results = pattern.findall(response)
	return [s.strip() for s in results]


In [ ]:
case_number = 200
max_length = 50
dimension_types = ['moral', 'personality', 'value'] # moral -> MFT, personality -> BigFive, value -> STBHV
description = None

for dimension_type in dimension_types:
    res = []
    if dimension_type == 'value':
        dimensions = list(schwartz_value_defination.keys())
        definitions = schwartz_value_defination
        description = schwartz_value_description
    elif dimension_type == 'moral':
        dimensions = list(moral_foundation_defination.keys())
        definitions = moral_foundation_defination
        description = moral_foundation_description
    else:
        dimensions = list(big5_defination.keys())
        definitions = big5_defination
        description = big5_value_description
    res_texts = {}
    for dimension in dimensions:
        print(f"Processing {dimension}...")
        start_time = time.time()
        prompt = get_prompt(dimension, description, case_number, max_length)
        message = [
            {'role': 'user', 'content': prompt},
        ]
        response = LLMFactory.process(message, model_name='GPT-4o', max_tokens=10000)
        end_time = time.time()
        print(f"Time cost: {end_time - start_time}s")
        time.sleep(max(60 - (end_time - start_time), 0))
        res_texts[dimension] = response

    final_res = []
    for dimension, response in res_texts.items():
        print(f"Dimension: {dimension}")
        extracted_sentences = extract_demonstration(response)
        print(f"Extracted {len(extracted_sentences)} sentences.")
        for i, sentence in enumerate(extracted_sentences):
            final_res.append({
                'dimension': dimension,
                'defination': definitions[dimension],
                'sentence': sentence
            })
    final_res_df = pd.DataFrame(final_res)
    final_res_df.to_csv(f'codes/data/reflection_data/{dimension_type}.csv', index=False)